In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date

import plotly.graph_objects as go

In [9]:
df = yf.download(['SPY','^SPXEW'],start = date(2022,1,1),end =date(2025,1,1))['Close']
df = np.log(df).diff()

df.var(axis = 0)

[*********************100%***********************]  2 of 2 completed


Ticker
SPY       0.000122
^SPXEW    0.000114
dtype: float64

In [5]:
sector_etfs = {
    'Baseline': {
        "Large-Cap": "SPY",
        "Small-Cap": "^SP600"
    },
    "Technology": {
        "Large-Cap": "XLK",
        "Small-Cap": "PSCT"
    },
    "Healthcare": {
        "Large-Cap": "XLV",
        "Small-Cap": "PSCH"
    },
    "Financials": {
        "Large-Cap": "XLF",
        "Small-Cap": "PSCF"
    },
    "Consumer Discretionary": {
        "Large-Cap": "XLY",
        "Small-Cap": "PSCD"
    },
    "Consumer Staples": {
        "Large-Cap": "XLP",
        "Small-Cap": "PSCC"
    },
    "Industrials": {
        "Large-Cap": "XLI",
        "Small-Cap": "PSCI"
    },
    "Energy": {
        "Large-Cap": "XLE",
        "Small-Cap": "PSCE"
    },
    "Materials": {
        "Large-Cap": "XLB",
        "Small-Cap": "PSCM"
    },
    "Utilities": {
        "Large-Cap": "XLU",
        "Small-Cap": "PSCU"
    },
    "Real Estate": {
        "Large-Cap": "XLRE",
        "Small-Cap": "ROOF"
    }
}

tickers = []
small_cap = []
large_cap = []
for sector, d in sector_etfs.items():
    t = [_ for _ in d.values()]
    tickers += t
    if sector != 'Baseline':
        large_cap.append(t[0])
        small_cap.append(t[1])

start = date(2020,1,1)
end = date(2025,1,1)

df = yf.download(tickers,start,end,auto_adjust=True)['Close']

df_ret = df.pct_change().dropna()

[*********************100%***********************]  22 of 22 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_5169/2254076443.py:63: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ret = df.pct_change().dropna()


In [16]:
start = date(2020,1,1)
end = date(2025,1,1)

df = yf.download(tickers,start,end,auto_adjust=True)['Close']

df_ret = df.pct_change().dropna()

[*********************100%***********************]  22 of 22 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_97370/2066423775.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ret = df.pct_change().dropna()


In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

# Define the sector ETFs (already provided in your code)
sector_etfs = {
    'Baseline': {
        "Large-Cap": "SPY",
        "Small-Cap": "^SP600"
    },
    "Technology": {
        "Large-Cap": "XLK",
        "Small-Cap": "PSCT"
    },
    "Healthcare": {
        "Large-Cap": "XLV",
        "Small-Cap": "PSCH"
    },
    "Financials": {
        "Large-Cap": "XLF",
        "Small-Cap": "PSCF"
    },
    "Consumer Discretionary": {
        "Large-Cap": "XLY",
        "Small-Cap": "PSCD"
    },
    "Consumer Staples": {
        "Large-Cap": "XLP",
        "Small-Cap": "PSCC"
    },
    "Industrials": {
        "Large-Cap": "XLI",
        "Small-Cap": "PSCI"
    },
    "Energy": {
        "Large-Cap": "XLE",
        "Small-Cap": "PSCE"
    },
    "Materials": {
        "Large-Cap": "XLB",
        "Small-CCap": "PSCM"
    },
    "Utilities": {
        "Large-Cap": "XLU",
        "Small-Cap": "PSCU"
    },
    "Real Estate": {
        "Large-Cap": "XLRE",
        "Small-Cap": "ROOF"
    }
}

# Initialize lists to store tickers
tickers = []
small_cap = []
large_cap = []
for sector, d in sector_etfs.items():
    t = [_ for _ in d.values()]
    tickers += t
    if sector != 'Baseline':
        large_cap.append(t[0])
        small_cap.append(t[1])

# Define the date range for data
start = date(2020, 1, 1)
end = date(2025, 1, 1)

# Download the adjusted closing prices
df = yf.download(tickers, start=start, end=end, auto_adjust=True)['Close']

# Calculate daily returns
df_ret = df.pct_change().dropna()

# Function to calculate 6-month price momentum
def calculate_momentum(df, period='60d'):
    # Calculate the momentum as the percentage change over the last 180 days (6 months)
    momentum = df.pct_change(periods=round(pd.Timedelta(period).days / 1)).iloc[-1]
    return momentum

# Create an empty list to store cumulative returns for the strategy
cumulative_returns = pd.Series(1, index=df_ret.index)

# Generate dates for rebalancing every 60 days
rebalance_dates = pd.date_range(start=df_ret.index[0], end=df_ret.index[-1], freq='60D')

# Loop through each rebalancing date to rebalance
for date in rebalance_dates:
    # Filter data up to the current rebalancing date
    current_data = df_ret.loc[:date]

    # Calculate the 6-month momentum for each ETF
    momentum_scores = calculate_momentum(current_data)

    # Create DataFrame for the momentum scores
    momentum_df = pd.DataFrame(momentum_scores)
    momentum_df.columns = ['Momentum']
    momentum_df['Rank'] = momentum_df['Momentum'].rank(ascending=False)

    # Select the top 3 Small-Cap ETFs for long positions and top 3 Large-Cap ETFs for short positions
    top_small_cap_etfs = momentum_df.loc[small_cap, 'Momentum'].nlargest(3).index.tolist()
    top_large_cap_etfs = momentum_df.loc[large_cap, 'Momentum'].nsmallest(3).index.tolist()

    # Calculate portfolio returns (long positions are positive, short positions are negative)
    long_returns = current_data[top_small_cap_etfs].mean(axis=1)  # Average returns of long ETFs
    short_returns = current_data[top_large_cap_etfs].mean(axis=1)  # Average returns of short ETFs

    # Compute portfolio returns (long minus short)
    portfolio_returns = long_returns - short_returns

    # Add portfolio returns to cumulative returns
    cumulative_returns.loc[portfolio_returns.index] *= (1 + portfolio_returns)

# Print final cumulative return
print(f"Final Cumulative Return of the Momentum Portfolio: {cumulative_returns[-1] * 100:.2f}%")


[*********************100%***********************]  22 of 22 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_97370/3218508952.py:73: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_97370/3218508952.py:112: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.00949701]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_97370/3218508952.py:115: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

Final Cumulative Return of the Momentum Portfolio: 100.00%


In [51]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y = cumulative_returns
    )
)

fig.show()